In [1]:
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds

epochs = 30
batch_size = 64
image_w, image_h = 32, 32

!rm -rf ./logs/mnist/*

In [2]:
(ds_train_raw, ds_test_raw), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=False,
    as_supervised=True,
    with_info=True,
)

n_classes = ds_info.features["label"].num_classes
n = ds_info.splits["train"].num_examples


def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = layers.Resizing(image_h, image_w)(image)
    image = tf.reshape(image, [-1])
    label = tf.one_hot(tf.cast(label, tf.int32), n_classes)
    label = tf.cast(label, tf.float32)
    return image, label


ds_train_normalized = ds_train_raw.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
).cache()

ds_test_normalized = ds_test_raw.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
).cache()


def prepare(ds, batch_size=batch_size):
    return ds.shuffle(n).batch(batch_size).prefetch(tf.data.AUTOTUNE)


dims = list(ds_train_normalized.take(1))[0][0].shape[0]

print("n: ", n, "n_classes: ", n_classes, "dims: ", dims)


def minmax_reducer(current, input):
    X, _ = input
    return (
        tf.reduce_min([current[0], X], axis=0),
        tf.reduce_max([current[0], X], axis=0),
    )


x0, _ = list(ds_train_normalized.take(1))[0]
min_train, max_train = ds_train_normalized.reduce((x0, x0), minmax_reducer)


n:  60000 n_classes:  10 dims:  1024


In [3]:
RandomFourierFeatures = keras.layers.experimental.RandomFourierFeatures

model_svm = keras.Sequential(
    [
        layers.Input(shape=(dims,)),
        RandomFourierFeatures(
            output_dim=2000, scale=10.0, kernel_initializer="gaussian"
        ),
        layers.Dense(units=n_classes),
    ]
)
model_svm.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.hinge,
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)

modeldir = "./logs/mnist/linear-8192-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_svm.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(
            log_dir=modeldir + "/log",
            histogram_freq=1,
        )
    ],
)
model_svm.save(modeldir + "/model")


Epoch 1/30
938/938 [==============================] - 7s 6ms/step - loss: 0.0661 - acc: 0.8933 - val_loss: 0.0414 - val_acc: 0.9371
Epoch 2/30
938/938 [==============================] - 4s 4ms/step - loss: 0.0354 - acc: 0.9478 - val_loss: 0.0290 - val_acc: 0.9573
Epoch 3/30
938/938 [==============================] - 4s 4ms/step - loss: 0.0293 - acc: 0.9572 - val_loss: 0.0289 - val_acc: 0.9560
Epoch 4/30
938/938 [==============================] - 4s 4ms/step - loss: 0.0257 - acc: 0.9637 - val_loss: 0.0267 - val_acc: 0.9596
Epoch 5/30
938/938 [==============================] - 5s 5ms/step - loss: 0.0230 - acc: 0.9675 - val_loss: 0.0269 - val_acc: 0.9630
Epoch 6/30
938/938 [==============================] - 4s 4ms/step - loss: 0.0219 - acc: 0.9696 - val_loss: 0.0218 - val_acc: 0.9675
Epoch 7/30
938/938 [==============================] - 4s 4ms/step - loss: 0.0206 - acc: 0.9710 - val_loss: 0.0263 - val_acc: 0.9664
Epoch 8/30
938/938 [==============================] - 4s 5ms/step - loss: 0.

INFO:tensorflow:Assets written to: ./logs/mnist/linear-8192-20230310-102730/model\assets


In [4]:
def gen_samples(psi, t=1000):
    return [
        list(
            ds_train_raw.shuffle(n)
            .take(psi)
            .map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(psi)
            .as_numpy_iterator()
        )[0][0]
        for _ in range(t)
    ]


def _tf_ann(X, samples, p=2, soft=True):
    m_dis = None
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


In [5]:
class IsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, soft=True, **kwargs):
        super(IsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p
        self.soft = soft

    def call(self, inputs):
        return _tf_ann(inputs, self.samples, self.p, self.soft)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
                "soft": self.soft,
            }
        )
        return config


def build_model(t_samples, p=2, soft=True):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        IsolationEncodingLayer(t_samples[i], p=p, soft=soft, name="ann_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [6]:
def _tf_ann_weighted(X, samples, sample_weights, p=2, soft=True):
    m_dis = None  # [n, psi]
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]  # [i, dims]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )  # [n, 1]
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    m_dis = m_dis * sample_weights

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


class FlexibleIsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, **kwargs):
        super(FlexibleIsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.sample_weights = self.add_weight(
            name="dimential_weights",
            shape=(
                1,
                self.samples.shape[0],
            ),
            initializer="ones",
            trainable=True,
        )
        super(FlexibleIsolationEncodingLayer, self).build(input_shape)

    def call(self, inputs):
        return _tf_ann_weighted(inputs, self.samples, self.sample_weights, self.p)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
            }
        )
        return config


def build_flex_model(t_samples, p=2):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        FlexibleIsolationEncodingLayer(t_samples[i], p=p, name="ann_flex_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [7]:
t_samples = gen_samples(psi=20, t=50)


model_hard_20_50 = build_model(t_samples, soft=False)
modeldir = "./logs/mnist/hard-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_20_50.save(modeldir + "/model")


Epoch 1/30
938/938 [==============================] - 116s 111ms/step - loss: 0.0871 - acc: 0.8589 - val_loss: 0.0481 - val_acc: 0.9199
Epoch 2/30
938/938 [==============================] - 96s 102ms/step - loss: 0.0456 - acc: 0.9220 - val_loss: 0.0407 - val_acc: 0.9312
Epoch 3/30
938/938 [==============================] - 99s 105ms/step - loss: 0.0398 - acc: 0.9326 - val_loss: 0.0378 - val_acc: 0.9343
Epoch 4/30
938/938 [==============================] - 97s 104ms/step - loss: 0.0368 - acc: 0.9383 - val_loss: 0.0364 - val_acc: 0.9371
Epoch 5/30
938/938 [==============================] - 97s 103ms/step - loss: 0.0349 - acc: 0.9417 - val_loss: 0.0356 - val_acc: 0.9404
Epoch 6/30
938/938 [==============================] - 98s 104ms/step - loss: 0.0335 - acc: 0.9447 - val_loss: 0.0352 - val_acc: 0.9398
Epoch 7/30
938/938 [==============================] - 97s 103ms/step - loss: 0.0325 - acc: 0.9462 - val_loss: 0.0350 - val_acc: 0.9418
Epoch 8/30
938/938 [==============================] - 

INFO:tensorflow:Assets written to: ./logs/mnist/hard-20x50-20230310-102955/model\assets


In [8]:
model_soft_20_50 = build_model(t_samples, soft=True)
modeldir = "./logs/mnist/soft-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_20_50.save(modeldir + "/model")


Epoch 1/30
938/938 [==============================] - 129s 121ms/step - loss: 0.1745 - acc: 0.8132 - val_loss: 0.0880 - val_acc: 0.8891
Epoch 2/30
938/938 [==============================] - 107s 114ms/step - loss: 0.0752 - acc: 0.9001 - val_loss: 0.0641 - val_acc: 0.9105
Epoch 3/30
938/938 [==============================] - 107s 115ms/step - loss: 0.0607 - acc: 0.9136 - val_loss: 0.0551 - val_acc: 0.9185
Epoch 4/30
938/938 [==============================] - 107s 114ms/step - loss: 0.0535 - acc: 0.9214 - val_loss: 0.0501 - val_acc: 0.9231
Epoch 5/30
938/938 [==============================] - 107s 114ms/step - loss: 0.0494 - acc: 0.9250 - val_loss: 0.0469 - val_acc: 0.9289
Epoch 6/30
938/938 [==============================] - 108s 115ms/step - loss: 0.0461 - acc: 0.9287 - val_loss: 0.0444 - val_acc: 0.9302
Epoch 7/30
938/938 [==============================] - 106s 113ms/step - loss: 0.0437 - acc: 0.9315 - val_loss: 0.0421 - val_acc: 0.9340
Epoch 8/30
938/938 [============================

INFO:tensorflow:Assets written to: ./logs/mnist/soft-20x50-20230310-111935/model\assets


In [9]:
model_flex_20_50 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/mnist/flex-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_20_50.save(modeldir + "/model")


Epoch 1/30
938/938 [==============================] - 153s 143ms/step - loss: 0.1681 - acc: 0.8169 - val_loss: 0.0798 - val_acc: 0.8873
Epoch 2/30
938/938 [==============================] - 131s 139ms/step - loss: 0.0674 - acc: 0.9004 - val_loss: 0.0573 - val_acc: 0.9108
Epoch 3/30
938/938 [==============================] - 129s 138ms/step - loss: 0.0532 - acc: 0.9153 - val_loss: 0.0493 - val_acc: 0.9212
Epoch 4/30
938/938 [==============================] - 128s 136ms/step - loss: 0.0473 - acc: 0.9233 - val_loss: 0.0453 - val_acc: 0.9238
Epoch 5/30
938/938 [==============================] - 131s 139ms/step - loss: 0.0441 - acc: 0.9273 - val_loss: 0.0421 - val_acc: 0.9290
Epoch 6/30
938/938 [==============================] - 128s 137ms/step - loss: 0.0414 - acc: 0.9314 - val_loss: 0.0397 - val_acc: 0.9345
Epoch 7/30
938/938 [==============================] - 128s 137ms/step - loss: 0.0398 - acc: 0.9343 - val_loss: 0.0395 - val_acc: 0.9351
Epoch 8/30
938/938 [============================

INFO:tensorflow:Assets written to: ./logs/mnist/flex-20x50-20230310-121413/model\assets


In [10]:
t_samples = gen_samples(psi=100, t=10)

model_hard_100_10 = build_model(t_samples, soft=False)
modeldir = "./logs/mnist/hard-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_100_10.save(modeldir + "/model")


Epoch 1/30
938/938 [==============================] - 143s 138ms/step - loss: 0.1494 - acc: 0.8464 - val_loss: 0.0564 - val_acc: 0.8970
Epoch 2/30
938/938 [==============================] - 122s 130ms/step - loss: 0.0529 - acc: 0.9017 - val_loss: 0.0476 - val_acc: 0.9112
Epoch 3/30
938/938 [==============================] - 123s 131ms/step - loss: 0.0465 - acc: 0.9120 - val_loss: 0.0439 - val_acc: 0.9188
Epoch 4/30
938/938 [==============================] - 123s 131ms/step - loss: 0.0432 - acc: 0.9174 - val_loss: 0.0420 - val_acc: 0.9224
Epoch 5/30
938/938 [==============================] - 123s 131ms/step - loss: 0.0410 - acc: 0.9209 - val_loss: 0.0409 - val_acc: 0.9237
Epoch 6/30
938/938 [==============================] - 123s 131ms/step - loss: 0.0395 - acc: 0.9239 - val_loss: 0.0400 - val_acc: 0.9258
Epoch 7/30
938/938 [==============================] - 123s 131ms/step - loss: 0.0383 - acc: 0.9262 - val_loss: 0.0395 - val_acc: 0.9273
Epoch 8/30
938/938 [============================

INFO:tensorflow:Assets written to: ./logs/mnist/hard-100x10-20230310-131846/model\assets


In [11]:
model_soft_100_10 = build_model(t_samples, soft=True)
modeldir = "./logs/mnist/soft-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_100_10.save(modeldir + "/model")


Epoch 1/30
938/938 [==============================] - 111s 104ms/step - loss: 0.1772 - acc: 0.8057 - val_loss: 0.0899 - val_acc: 0.8839
Epoch 2/30
938/938 [==============================] - 86s 91ms/step - loss: 0.0771 - acc: 0.8931 - val_loss: 0.0668 - val_acc: 0.9051
Epoch 3/30
938/938 [==============================] - 84s 90ms/step - loss: 0.0621 - acc: 0.9088 - val_loss: 0.0562 - val_acc: 0.9187
Epoch 4/30
938/938 [==============================] - 85s 91ms/step - loss: 0.0550 - acc: 0.9175 - val_loss: 0.0511 - val_acc: 0.9229
Epoch 5/30
938/938 [==============================] - 86s 92ms/step - loss: 0.0503 - acc: 0.9228 - val_loss: 0.0469 - val_acc: 0.9282
Epoch 6/30
938/938 [==============================] - 87s 93ms/step - loss: 0.0474 - acc: 0.9276 - val_loss: 0.0442 - val_acc: 0.9297
Epoch 7/30
938/938 [==============================] - 87s 92ms/step - loss: 0.0448 - acc: 0.9305 - val_loss: 0.0427 - val_acc: 0.9314
Epoch 8/30
938/938 [==============================] - 88s 94

INFO:tensorflow:Assets written to: ./logs/mnist/soft-100x10-20230310-142137/model\assets


In [12]:
model_flex_100_10 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/mnist/flex-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_100_10.save(modeldir + "/model")


Epoch 1/30
938/938 [==============================] - 148s 143ms/step - loss: 0.1703 - acc: 0.8085 - val_loss: 0.0809 - val_acc: 0.8896
Epoch 2/30
938/938 [==============================] - 127s 135ms/step - loss: 0.0685 - acc: 0.8951 - val_loss: 0.0593 - val_acc: 0.9057
Epoch 3/30
938/938 [==============================] - 130s 139ms/step - loss: 0.0548 - acc: 0.9107 - val_loss: 0.0500 - val_acc: 0.9193
Epoch 4/30
938/938 [==============================] - 128s 136ms/step - loss: 0.0484 - acc: 0.9201 - val_loss: 0.0460 - val_acc: 0.9213
Epoch 5/30
938/938 [==============================] - 153s 163ms/step - loss: 0.0452 - acc: 0.9229 - val_loss: 0.0437 - val_acc: 0.9262
Epoch 6/30
938/938 [==============================] - 127s 135ms/step - loss: 0.0428 - acc: 0.9277 - val_loss: 0.0413 - val_acc: 0.9283
Epoch 7/30
938/938 [==============================] - 128s 136ms/step - loss: 0.0410 - acc: 0.9302 - val_loss: 0.0400 - val_acc: 0.9302
Epoch 8/30
938/938 [============================

INFO:tensorflow:Assets written to: ./logs/mnist/flex-100x10-20230310-150555/model\assets
